In [ ]:
import albumentations as A
import cv2
import os

def read_yolo_labels(label_path):
    """Read YOLO format labels from a file."""
    with open(label_path, 'r') as f:
        labels = f.readlines()
    bboxes = []
    class_labels = []
    for label in labels:
        parts = label.strip().split()
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        bboxes.append([x_center, y_center, width, height])
        class_labels.append(class_id)
    return bboxes, class_labels

def write_yolo_labels(label_path, bboxes):
    """Write YOLO format labels to a file."""
    with open(label_path, 'w') as f:
        for bbox in bboxes:
            x_center = round(bbox[0], 6)
            y_center = round(bbox[1], 6)
            width = round(bbox[2], 6)
            height = round(bbox[3], 6)
            f.write(f"0 {x_center} {y_center} {width} {height}\n")

def main(input_image_dir, input_label_dir, output_image_dir, output_label_dir, augmentations_per_image):
    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)
    if not os.path.exists(output_label_dir):
        os.makedirs(output_label_dir)
    
    for image_name in os.listdir(input_image_dir):
        if image_name.endswith(('.jpg', '.png')):
            image_path = os.path.join(input_image_dir, image_name)
            label_name = os.path.splitext(image_name)[0] + '.txt'
            label_path = os.path.join(input_label_dir, label_name)

            # Read image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error reading image {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Read labels
            bboxes, class_labels = read_yolo_labels(label_path)
            
            for i in range(augmentations_per_image):
                # Apply data augmentation
                transformed = transform(image=image, bboxes=bboxes)
                transformed_image = transformed['image']
                transformed_bboxes = transformed['bboxes']

                # Generate output paths
                output_image_path = os.path.join(output_image_dir, os.path.splitext(image_name)[0] + f'_aug_{i}.jpg')
                output_label_path = os.path.join(output_label_dir, os.path.splitext(image_name)[0] + f'_aug_{i}.txt')

                # Save augmented image and labels
                cv2.imwrite(output_image_path, cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR))
                write_yolo_labels(output_label_path, transformed_bboxes)

In [ ]:
transform = A.Compose([
	    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
	    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
	    A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.5),
	    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
	    A.GaussianBlur(blur_limit=(3, 7), p=0.5),
	    A.RandomRotate90(p=0.5),
	    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.5)
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.1))

In [ ]:
input_image_dir = '/mnt/e/train/images'
input_label_dir = '/mnt/e/train/labels'
output_image_dir = '/mnt/e/augmented-train/images'
output_label_dir = '/mnt/e/augmented-train/labels'

main(input_image_dir, input_label_dir, output_image_dir, output_label_dir, augmentations_per_image=5)